In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
train_epochs = 1
batch_size = 100
test_batch_size = 100
learning_rate = 0.01
display_step = 10

In [3]:
(x_train_data, y_train_data), (x_test_data, y_test_data) = keras.datasets.cifar10.load_data()

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data)) \
    .map(lambda a, b: (a / 255, tf.one_hot(b, 10))) \
    .shuffle(1000).repeat(train_epochs).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data, y_test_data)) \
    .map(lambda a, b: (a / 255, tf.one_hot(b, 10))) \
    .shuffle(1000).repeat().batch(test_batch_size)

train_iterator = train_dataset.make_initializable_iterator()
train_next_element = train_iterator.get_next()
test_iterator = test_dataset.make_initializable_iterator()
test_next_element = test_iterator.get_next()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
x = tf.placeholder(tf.float32, [None, 32, 32, 3], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

In [6]:
conv1 = tf.layers.conv2d(inputs=x,filters=32,kernel_size=[5, 5],strides=1,padding='same',activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2, 2],strides=2)
conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],strides=1,padding='same',activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2, 2],strides=2)
flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
dense = tf.layers.dense(inputs=flat, units=128, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.5)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.


In [7]:
pred = tf.layers.dense(inputs=dropout, units=10)

In [8]:
loss_function = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)

Instructions for updating:
Use tf.cast instead.


In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [10]:
correct_pred = tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(train_iterator.initializer)
sess.run(test_iterator.initializer)

In [12]:
step = 0
while 1:
    try:
        xs, ys = sess.run(train_next_element)
    except tf.errors.OutOfRangeError:
        break

    sess.run([optimizer], feed_dict={x: xs, y: ys})
    step += 1
    if step % display_step == 0:
        xss, yss = sess.run(test_next_element)
        loss, acc = sess.run([loss_function, accuracy], feed_dict={x: xss, y: yss})
        print('step:{:d} loss:{:.05f} acc:{:.05f}'.format(step, loss, acc))


step:10 loss:2.30570 acc:0.09000
step:20 loss:2.28180 acc:0.14000
step:30 loss:2.21219 acc:0.19000
step:40 loss:2.16115 acc:0.20000
step:50 loss:1.98060 acc:0.27000
step:60 loss:2.02055 acc:0.26000
step:70 loss:1.97656 acc:0.29000
step:80 loss:2.03214 acc:0.27000
step:90 loss:1.89747 acc:0.39000
step:100 loss:1.94737 acc:0.32000
step:110 loss:2.04442 acc:0.22000
step:120 loss:1.83090 acc:0.36000
step:130 loss:1.86879 acc:0.32000
step:140 loss:1.88326 acc:0.26000
step:150 loss:1.71838 acc:0.36000
step:160 loss:1.84461 acc:0.30000
step:170 loss:1.98106 acc:0.25000
step:180 loss:1.78292 acc:0.31000
step:190 loss:1.93423 acc:0.30000
step:200 loss:1.97306 acc:0.24000
step:210 loss:1.86205 acc:0.31000
step:220 loss:1.86340 acc:0.37000
step:230 loss:1.69379 acc:0.36000
step:240 loss:1.81818 acc:0.35000
step:250 loss:2.02534 acc:0.29000
step:260 loss:1.84191 acc:0.29000
step:270 loss:1.85498 acc:0.34000
step:280 loss:1.67842 acc:0.37000
step:290 loss:1.90287 acc:0.29000
step:300 loss:1.80552 a